# Basic usage of the ``h5pywrappers`` library #

## A NOTE BEFORE STARTING ##

Since the ``h5pywrappers`` git repository tracks this notebook under its
original basename ``basic_usage.ipynb``, we recommend that you copy the original
notebook and rename it to any other basename that is not one of the original
basenames that appear in the ``<root>/examples`` directory before executing any
of the notebook cells below, where ``<root>`` is the root of the
``h5pywrappers`` repository. This way you can explore the notebook by executing
and modifying cells without changing the original notebook, which is being
tracked by git.

## Table of contents ##

- [Import necessary modules](#Import-necessary-modules)
- [Introduction](#Introduction)
- [Using the ``h5pywrappers`` function](#Using-the-h5pywrappers-function)

## Import necessary modules ##

In [30]:
# For general array handling and constructing random number generators.
import numpy as np

# For converting objects.
import czekitout.convert



# The library that is the subject of this demonstration.
import h5pywrappers

## Introduction ##

In this notebook, we demonstrate how one can use each function and class in the
``h5pywrappers`` library.

In order to execute the cells in this notebook as intended, a set of Python
libraries need to be installed in the Python environment within which the cells
of the notebook are to be executed. For this particular notebook, users need to
install:

    h5pywrappers
    jupyter

Users can install these libraries either via `pip`:

    pip install h5pywrappers[examples]

or `conda`:

    conda install -y h5pywrappers jupyter -c conda-forge

You can find the documentation for the ``h5pywrappers`` library
[here](https://mrfitzpa.github.io/h5pywrappers/_autosummary/h5pywrappers.html).
It is recommended that you consult the documentation of this library as you
explore the notebook. Moreover, users should execute the cells in the order that
they appear, i.e. from top to bottom, as some cells reference variables that are
set in other cells above them. **Users should make sure to navigate the
documentation for the version of ``h5pywrappers`` that they are currently
using.**

## Using the ``h5pywrappers`` function ##

Let's create an HDF5 file and populate it with HDF5 data.

In [2]:
kwargs = {"filename": "file_1.h5", "path_in_file": "/group_A/dataset_A"}
obj_id = h5pywrappers.obj.ID(**kwargs)

float_array = np.random.rand(2, 5, 4, 6)

kwargs = {"dataset": float_array, "dataset_id": obj_id, "write_mode": "w"}
h5pywrappers.dataset.save(**kwargs)

The above notebook cell will create a new file at the file path 
``"./file_1.h5"``, with any file at the same file path being replaced by the new
file. In the new file, an HDF5 dataset is located at the HDF5 path 
``"/group_A/dataset_A"``.

If we try to re-save the HDF5 dataset when ``write_mode == "w-"``, an exception
will be raised. The following notebook cell should raise an exception:

In [3]:
kwargs = {"dataset": float_array, "dataset_id": obj_id, "write_mode": "w-"}
h5pywrappers.dataset.save(**kwargs)

OSError: Cannot save the dataset to the HDF5 path ``'/group_A/dataset_A'`` of the HDF5 file at the file path ``'file_1.h5'`` because an HDF5 file already exists at said file path and the parameter ``write_mode`` was set to ``'w-'``, which prohibits modifying any such pre-existing file.

Let's now load the HDF5 dataset that we just successfully saved, but in 
read-only mode.

In [4]:
kwargs = {"dataset_id": obj_id, "read_only": True}
dataset = h5pywrappers.dataset.load(**kwargs)
dataset

<HDF5 dataset "dataset_A": shape (2, 5, 4, 6), type "<f8">

Let's check that the data is correct.

In [5]:
bool(np.all(dataset[()] == float_array))

True

Since we are in read-only mode, we cannot modify the HDF5 dataset that we 
loaded. The following notebook cell should raise an exception:

In [6]:
dataset[0, 0, 0, 0] = 0

OSError: Can't synchronously write data (no write intent on file)

Let's create a new HDF5 file and populate it with the same HDF5 dataset of the
previous HDF5 file.

In [7]:
kwargs = {"filename": "file_2.h5", "path_in_file": "/group_A/dataset_A"}
obj_id = h5pywrappers.obj.ID(**kwargs)

kwargs = {"dataset": dataset, "dataset_id": obj_id, "write_mode": "w"}
h5pywrappers.dataset.save(**kwargs)

Let's now close the first HDF5 file.

In [8]:
dataset.file.close()

Let's load the HDF5 dataset in the second HDF5 file, modify it, and then close 
the file.

In [9]:
kwargs = {"dataset_id": obj_id, "read_only": False}
dataset = h5pywrappers.dataset.load(**kwargs)
dataset[0, 0, 0, 0] += 1
dataset.file.close()

We can also store strings as HDF5 datasets. Let's append such an HDF5 dataset to
the first HDF5 file.

In [10]:
kwargs = {"filename": "file_1.h5", 
          "path_in_file": "/group_B/dataset_B"}
obj_id = h5pywrappers.obj.ID(**kwargs)

string = "hello world"

kwargs = {"dataset": string, "dataset_id": obj_id, "write_mode": "a"}
h5pywrappers.dataset.save(**kwargs)

Let's load the new HDF5 dataset.

In [11]:
kwargs = {"dataset_id": obj_id, "read_only": True}
dataset = h5pywrappers.dataset.load(**kwargs)
dataset

<HDF5 dataset "dataset_B": shape (), type "|O">

Let's check that the contents of the HDF5 file are correct.

In [12]:
kwargs = {"obj": dataset[()], "obj_name": "dataset"}
czekitout.convert.to_str_from_str_like(**kwargs)

'hello world'

Let's now close the HDF5 file.

In [13]:
dataset.file.close()

The last optional write mode for HDF5 datasets is toggled by setting
``write_mode`` to ``"a-"``. In this mode, the HDF5 dataset of interest is saved
unless an HDF5 object already exists at the target HDF5 path of the target HDF5
file, in which case an error is raised. The following notebook cell should raise
an exception:

In [14]:
kwargs = {"dataset": string, "dataset_id": obj_id, "write_mode": "a-"}
h5pywrappers.dataset.save(**kwargs)

OSError: Cannot save the dataset to the HDF5 path ``'/group_B/dataset_B'`` of the HDF5 file at the file path ``'file_1.h5'`` because an HDF5 object already exists there and the parameter ``write_mode`` was set to ``'a-'``, which prohibits replacing any such pre-existing HDF5 object.

Let's add an HDF5 attribute to one of the HDF5 groups in the first HDF5 file.

In [15]:
kwargs = {"filename": "file_1.h5", "path_in_file": "/group_A"}
obj_id = h5pywrappers.obj.ID(**kwargs)

attr = "foobar"
attr_name = "alternate_group_name"

kwargs = {"obj_id": obj_id, "attr_name": attr_name}
attr_id = h5pywrappers.attr.ID(**kwargs)

kwargs = {"attr": "foobar", "attr_id": attr_id, "write_mode": "a"}
h5pywrappers.attr.save(**kwargs)

Note that users can also save HDF5 attributes in the write mode ``"a-"``.

Let's load the HDF5 attribute and check that it is set to the correct value.
Note that there is no need to close the HDF5 file as it is done automatically in
this case.

In [16]:
kwargs = {"attr_id": attr_id}
h5pywrappers.attr.load(**kwargs)

'foobar'

Let's create a new empty HDF5 group in the first HDF5 file.

In [17]:
kwargs = {"filename": "file_1.h5", "path_in_file": "/group_C"}
obj_id = h5pywrappers.obj.ID(**kwargs)

kwargs = {"group": None, "group_id": obj_id, "write_mode": "a"}
h5pywrappers.group.save(**kwargs)

Note that the same writing modes are available for HDF5 groups as those of HDF5
datasets.

Let's load the newly created HDF5 group.

In [18]:
kwargs = {"group_id": obj_id, "read_only": True}
group = h5pywrappers.group.load(**kwargs)
group

<HDF5 group "/group_C" (0 members)>

Let's save the newly created HDF5 group to the second HDF5 file.

In [19]:
kwargs = {"filename": "file_2.h5", "path_in_file": "/group_B"}
obj_id = h5pywrappers.obj.ID(**kwargs)

kwargs = {"group": group, "group_id": obj_id, "write_mode": "a-"}
h5pywrappers.group.save(**kwargs)

Let's close the first HDF5 file.

In [20]:
group.file.close()

Let's save an HDF5 "scalar" to the first HDF5 file. By "scalar", we mean a
single numerical data element.

In [21]:
kwargs = {"filename": "file_1.h5", "path_in_file": "/group_C/scalar_C"}
obj_id = h5pywrappers.obj.ID(**kwargs)

scalar = 2.3+0.5j

kwargs = {"scalar": scalar, "scalar_id": obj_id, "write_mode": "a-"}
h5pywrappers.scalar.save(**kwargs)

Note that the same writing modes are available for HDF5 scalars as those of HDF5
datasets.

Let's load the HDF5 scalar and check that the correct value was stored. Note 
that there is no need to close the HDF5 file as it is done automatically in this
case.

In [22]:
kwargs = {"scalar_id": obj_id}
h5pywrappers.scalar.load(**kwargs)

(2.3+0.5j)

Let's save an HDF5 "datasubset" to the first HDF5 file. By "datasubset", we mean
an array obtained by taking a multidimensional slice of an HDF5 dataset in an
HDF5 file. In other words, let's modify a multidimensional slice of one of the
previously saved HDF5 datasets.

In [23]:
kwargs = {"filename": "file_1.h5", "path_in_file": "/group_A/dataset_A"}
obj_id = h5pywrappers.obj.ID(**kwargs)

multi_dim_slice = (1, slice(1, None, 1), [3, 0], slice(None))

kwargs = {"dataset_id": obj_id, "multi_dim_slice": multi_dim_slice}
datasubset_id = h5pywrappers.datasubset.ID(**kwargs)

float_array = np.random.rand(2, 4, 6)

kwargs = {"datasubset": float_array, "datasubset_id": datasubset_id}
h5pywrappers.datasubset.save(**kwargs)

Let's load the HDF5 datasubset and check that its contents are correct. Note 
that there is no need to close the HDF5 file as it is done automatically in this
case.

In [24]:
kwargs = {"datasubset_id": datasubset_id}
bool(np.all(h5pywrappers.datasubset.load(**kwargs) == float_array))

True

Note that users can use alternatively ``h5pywrappers.obj.load`` to load any HDF5
group or dataset. Let's try loading an HDF5 dataset from the first file using 
this alternative loading function.

In [25]:
kwargs = {"obj_id": obj_id, "read_only": True}
dataset = h5pywrappers.obj.load(**kwargs)
dataset

<HDF5 dataset "dataset_A": shape (2, 5, 4, 6), type "<f8">

Let's close the first HDF5 file.

In [26]:
dataset.file.close()

One of the features of ``h5pywrappers`` is that the classes 
``h5pywrappers.obj.ID``, ``h5pywrappers.attr.ID``, and
``h5pywrappers.datasubset.ID`` are subclasses of
``fancytypes.PreSerializableAndUpdatable``, which is a special class that is
both updatable, and "pre-serializable". See [here](https://mrfitzpa.github.io/fancytypes/_autosummary/fancytypes.PreSerializableAndUpdatable.html), for a
description of the class ``fancytypes.PreSerializableAndUpdatable``, and a
definition of "pre-serialization" (as well as de-pre-serialization).

Let's pre-serialize ``datasubset_id`` as a demonstration of pre-serialization.

In [27]:
json_document = datasubset_id.pre_serialize()
json_document

{'dataset_id': {'filename': 'file_1.h5', 'path_in_file': '/group_A/dataset_A'},
 'multi_dim_slice': (1,
  {'start': 1, 'stop': None, 'step': 1},
  [3, 0],
  {'start': None, 'stop': None, 'step': None})}

Let's save the above JSON document ``json_document`` to the first HDF5 file.

In [28]:
kwargs = {"filename": "file_1.h5", "path_in_file": "/group_D/json_document_D"}
obj_id = h5pywrappers.obj.ID(**kwargs)

kwargs = {"json_document": json_document, 
          "json_document_id": obj_id, 
          "write_mode": "a-"}
h5pywrappers.json.document.save(**kwargs)

Note that the same writing modes are available for JSON document as those of 
HDF5 datasets.

Let's load the JSON document and check that its contents are correct. Note that
there is no need to close the HDF5 file as it is done automatically in this 
case.

In [29]:
kwargs = {"json_document_id": obj_id}
h5pywrappers.json.document.load(**kwargs)

{'dataset_id': {'filename': 'file_1.h5', 'path_in_file': '/group_A/dataset_A'},
 'multi_dim_slice': [1,
  {'start': 1, 'stop': None, 'step': 1},
  [3, 0],
  {'start': None, 'stop': None, 'step': None}]}